# Importation des librairies

In [62]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import datetime as dt
import scipy.stats as st
import matplotlib.pyplot as plt

## Chargement et affichage des donnees

In [63]:
data = pd.read_csv("operations.csv")
data

,date_operation,libelle,montant,solde_avt_ope,categ
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT
3,2023-04-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,1416.81,AUTRE
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,2092.81,AUTRE
...,...,...,...,...,...
304,2023-10-05,CARTE XX XX XX XX XX XX,-10.64,2440.94,AUTRE
305,2023-10-05,CARTE XX XX XX XX,-4.80,2430.30,AUTRE
306,2023-10-06,FORFAIT COMPTE SUPERBANK XX XX XX XX,-1.92,2425.50,COTISATION BANCAIRE
307,2023-10-06,CARTE XX XX CHEZ LUC XX,-10.00,2423.58,RESTAURANT


# Detection des erreurs

### Erreur de type

In [64]:
data.dtypes

date_operation     object
libelle            object
montant           float64
solde_avt_ope     float64
categ              object
dtype: object

#### Conversion de la dates

In [65]:
data['date_operation'] = pd.to_datetime(data['date_operation'])

In [66]:
data.dtypes

date_operation    datetime64[ns]
libelle                   object
montant                  float64
solde_avt_ope            float64
categ                     object
dtype: object

### Valeur manquantes

In [67]:
data.isna().sum()

date_operation    0
libelle           0
montant           2
solde_avt_ope     0
categ             1
dtype: int64

#### Variables ayant les valeurs manquantes

In [68]:
mv = data.isna().sum()
mv[mv>0]

montant    2
categ      1
dtype: int64

#### Individus ayant les valeurs manquantes

In [69]:
data.loc[data['montant'].isnull(),:]

,date_operation,libelle,montant,solde_avt_ope,categ
107,2023-06-12,CARTE XX XX LES ANCIENS ROBINSON XX,NaN,4667.19,COURSES
269,2023-09-11,CARTE XX XX XX XX,NaN,3401.93,AUTRE


In [70]:
data.loc[data['categ'].isnull(),:]

,date_operation,libelle,montant,solde_avt_ope,categ
156,2023-07-06,PRELEVEMENT XX TELEPHONE XX XX,-36.48,3295.68,NaN


In [71]:
montant_na = data.loc[data['montant'].isnull(),:]

for index in montant_na.index:
    data.loc[index, 'montant'] = data.loc[index+1, 'solde_avt_ope'] - data.loc[index, 'solde_avt_ope']

In [72]:
data.loc[data['montant'].isnull(),:]

,date_operation,libelle,montant,solde_avt_ope,categ


#### Individus ayant les cat identique

In [74]:
data.loc[data['libelle'] == 'PRELEVEMENT XX TELEPHONE XX XX',:]

,date_operation,libelle,montant,solde_avt_ope,categ
8,2023-04-05,PRELEVEMENT XX TELEPHONE XX XX,-7.02,2056.02,FACTURE TELEPHONE
62,2023-05-09,PRELEVEMENT XX TELEPHONE XX XX,-7.02,4090.10,FACTURE TELEPHONE
102,2023-06-07,PRELEVEMENT XX TELEPHONE XX XX,-6.38,4688.91,FACTURE TELEPHONE
156,2023-07-06,PRELEVEMENT XX TELEPHONE XX XX,-36.48,3295.68,NaN
204,2023-08-07,PRELEVEMENT XX TELEPHONE XX XX,-7.46,3751.73,FACTURE TELEPHONE
260,2023-09-05,PRELEVEMENT XX TELEPHONE XX XX,-6.38,3453.96,FACTURE TELEPHONE
308,2023-10-06,PRELEVEMENT XX TELEPHONE XX XX,-13.58,2413.58,FACTURE TELEPHONE


In [75]:
data.loc[data['categ'].isnull(), 'categ'] = 'FACTURE TELEPHONE'

### Recherche des doublons

In [76]:
data.loc[data[['date_operation', 'libelle', 'montant', 'solde_avt_ope']].duplicated(keep=False),:]

,date_operation,libelle,montant,solde_avt_ope,categ
43,2023-04-25,CARTE XX XX LES ANCIENS ROBINSON XX,-32.67,3647.67,COURSES
44,2023-04-25,CARTE XX XX LES ANCIENS ROBINSON XX,-32.67,3647.67,COURSES


#### Traitement des doublons

In [77]:
data.drop_duplicates(subset=['date_operation', 'libelle', 'montant', 'solde_avt_ope'],inplace=True,ignore_index=True)

### Dectection des outliers

In [78]:
data.describe(exclude=object)

,date_operation,montant,solde_avt_ope
count,308,308.000000,308.000000
mean,2023-07-05 10:59:13.246753280,-45.782013,3395.301071
min,2023-03-31 00:00:00,-15000.000000,1416.810000
25%,2023-05-21 06:00:00,-20.447500,3010.737500
50%,2023-07-05 12:00:00,-9.600000,3452.465000
75%,2023-08-21 00:00:00,-2.715000,3787.232500
max,2023-10-06 00:00:00,1071.600000,4709.310000
std,NaN,872.818105,667.109412


In [83]:
t = data.loc[data['montant'] == -15000,:].index[0]
data.iloc[t-1:t+2,:]

,date_operation,libelle,montant,solde_avt_ope,categ
197,2023-08-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,3121.35,AUTRE
198,2023-08-03,CARTE XX XX XX XX,-15000.00,3797.35,AUTRE
199,2023-08-03,CARTE XX XX L'EPICERIE DEMBAS XX XX,-10.51,3782.96,AUTRE


In [88]:
data.loc[data['montant'] == -15000, 'montant'] = -14.39

In [89]:
data.describe()

,date_operation,montant,solde_avt_ope
count,308,308.000000,308.000000
mean,2023-07-05 10:59:13.246753280,2.872565,3395.301071
min,2023-03-31 00:00:00,-602.270000,1416.810000
25%,2023-05-21 06:00:00,-20.047500,3010.737500
50%,2023-07-05 12:00:00,-9.600000,3452.465000
75%,2023-08-21 00:00:00,-2.715000,3787.232500
max,2023-10-06 00:00:00,1071.600000,4709.310000
std,NaN,176.089858,667.109412


## Detection des doublons

In [85]:
data

,date_operation,libelle,montant,solde_avt_ope,categ
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT
3,2023-04-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,1416.81,AUTRE
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,2092.81,AUTRE
...,...,...,...,...,...
303,2023-10-05,CARTE XX XX XX XX XX XX,-10.64,2440.94,AUTRE
304,2023-10-05,CARTE XX XX XX XX,-4.80,2430.30,AUTRE
305,2023-10-06,FORFAIT COMPTE SUPERBANK XX XX XX XX,-1.92,2425.50,COTISATION BANCAIRE
306,2023-10-06,CARTE XX XX CHEZ LUC XX,-10.00,2423.58,RESTAURANT


In [127]:
cat_data = []
num_data = []

In [128]:
for i, c in enumerate(data.dtypes):
    if c == 'object':
        cat_data.append(data.iloc[:,i])
    else:
        num_data.append(data.iloc[:,i])

In [129]:
cat_data = pd.DataFrame(cat_data).T
num_data = pd.DataFrame(num_data).T

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>)

In [ ]:
num_data.dtypes

In [ ]:
Q1,Q3 = np.percentile(num_data, [25,75])
IQR = Q3 - Q1
outlier = []
for item in num_data:
    if item<(Q1 - 1.5 * IQR) or item>(Q3 + 1.5 * IQR):
        outlier.append(item)
outlier

In [ ]:
oulie